In [1]:
import pandas as pd
import numpy as np

In [2]:
# Try reading the file with shift_jis encoding
data_shift_jis = pd.read_csv('../data/在庫推移9月.csv', encoding='shift_jis')
data_shift_jis.head()

,Unnamed: 0,計測日時,品番,拠点所番地,在庫数（箱）,入庫数（箱）,出庫数（箱）
0,0,2023/09/04 00:00:00,019120LC030,30113,12.0,0.0,0.0
1,1,2023/09/04 00:00:00,019120LC040,10114,3.0,0.0,0.0
2,2,2023/09/04 00:00:00,019120LC050,20115,4.0,0.0,0.0
3,3,2023/09/04 00:00:00,019120LC060,30116,9.0,0.0,0.0
4,4,2023/09/04 00:00:00,019120LC070,20117,9.0,0.0,0.0


In [3]:
#以下の手順でデータを処理します：
#計測日時を日付として解釈し、それを基にデータを日付毎にグループ化します。
#各グループ内で、品番毎の入庫数、出庫数を合計します。
#各日付の最後のレコードの在庫数を使用して、その日の在庫数を取得します。

# Convert 計測日時 to datetime and extract only the date
data_shift_jis['計測日'] = pd.to_datetime(data_shift_jis['計測日時']).dt.date

# Group by 計測日 and 品番
grouped = data_shift_jis.groupby(['計測日', '品番'])

# Calculate the sum of 入庫数（箱） and 出庫数（箱） for each group
in_out_sum = grouped[['入庫数（箱）', '出庫数（箱）']].sum()

# Get the last record's 在庫数（箱） for each group (as the end-of-day inventory)
end_of_day_inventory = grouped['在庫数（箱）'].last()

# Combine the results
result = pd.concat([in_out_sum, end_of_day_inventory], axis=1)
result = result.reset_index()

# Extract only the day from the '検収日' column
result['計測日'] = pd.to_datetime(result['計測日']).dt.day

result.columns = ['計測日', '品番', '入庫数（箱）', '出庫数（箱）', '在庫数（箱）']
result.head()

,計測日,品番,入庫数（箱）,出庫数（箱）,在庫数（箱）
0,4,019120LC030,5.0,2.0,15.0
1,4,019120LC040,1.0,0.0,4.0
2,4,019120LC050,1.0,0.0,5.0
3,4,019120LC060,2.0,1.0,10.0
4,4,019120LC070,2.0,1.0,10.0


In [4]:
# Randomly select a product code (品番)
random_product_code = result['品番'].sample(1).iloc[0]
# Extract data for the selected product code
selected_product_data = result[result['品番'] == random_product_code]
selected_product_data

,計測日,品番,入庫数（箱）,出庫数（箱）,在庫数（箱）
178,4,9030111A014,3.0,2.0,6.0
503,5,9030111A014,5.0,5.0,6.0
828,6,9030111A014,5.0,4.0,7.0
1153,7,9030111A014,3.0,5.0,5.0
1478,8,9030111A014,6.0,4.0,7.0
1801,11,9030111A014,3.0,2.0,7.0
2122,12,9030111A014,5.0,6.0,6.0
2443,13,9030111A014,4.0,4.0,6.0
2764,14,9030111A014,5.0,5.0,6.0
3085,15,9030111A014,5.0,4.0,7.0


In [7]:
# Load the new uploaded file with shift_jis encoding
kanban_data = pd.read_csv('../data/20231011作成_所在管理LT（9月納入）.csv', encoding='shift_jis')
kanban_data.head()

c:\users\1082794-z100\.pyenv\pyenv-win\versions\3.7.9\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,かんばんシリアル,伝票番号,拠点所番地,品番,品名,収容数,仕入先名,仕入先工場名,ステータス,納入日,...,回収日時,発注取消日時,発注〜印刷LT,発注〜検収LT,発注〜順立装置入庫LT,発注〜順立装置出庫LT,発注〜組立LT,発注〜回収LT,更新日時,長期滞留フラグ
0,1Z23H00087251,XCEM680,40427,35771ECE010,ｼｬﾌﾄｱｳﾄﾌﾟｯﾄ,6,アイシン機工（株）,吉良工場,回収済,2023/9/1,...,2023/9/1 16:51,< NULL >,0.32,1.15,1.2,1.3,< NULL >,1.41,2023/9/4 13:23,< NULL >
1,1Z23H00087250,XCEM680,40427,35771ECE010,ｼｬﾌﾄｱｳﾄﾌﾟｯﾄ,6,アイシン機工（株）,吉良工場,回収済,2023/9/1,...,2023/9/1 16:51,< NULL >,0.32,1.15,1.2,1.22,< NULL >,1.41,2023/9/4 13:23,< NULL >
2,1Z23H00062638,XCEF320,30356,35882ECB010,ﾁｭｰﾌﾞﾃﾞﾌｷﾞﾔﾘｭｰﾌﾞｱﾌﾟﾗｲ,50,（株）メタルテック,< NULL >,回収済,2023/9/1,...,2023/9/1 16:51,< NULL >,0.08,2.01,2.17,2.31,< NULL >,2.41,2023/9/7 13:23,< NULL >
3,1Z23H00061227,XCEF320,30357,35847ECE010,ﾁｭｰﾌﾞT/Aﾙｰﾌﾞｱﾌﾟﾗｲ,40,（株）メタルテック,< NULL >,回収済,2023/9/1,...,2023/9/1 16:51,< NULL >,0.08,2.01,2.17,2.19,< NULL >,2.41,2023/9/7 13:23,< NULL >
4,1Z23H00087448,XCEM680,40427,35771ECE010,ｼｬﾌﾄｱｳﾄﾌﾟｯﾄ,6,アイシン機工（株）,吉良工場,回収済,2023/9/1,...,2023/9/1 17:38,< NULL >,0.32,1.15,1.2,1.31,< NULL >,1.44,2023/9/4 13:23,< NULL >


In [8]:
#手順を以下のように進めます：
#"検収日時（HH:MM）" から日付情報を抽出します。
#品番と検収日でグループ化し、各グループの行数（検収数）をカウントします。

# Convert 検収日時（HH:MM） to datetime and extract only the date
kanban_data['検収日'] = pd.to_datetime(kanban_data['検収日時'], errors='coerce').dt.date

# Group by 検収日 and 品番, then count the number of rows for each group
kanban_grouped = kanban_data.groupby(['検収日', '品番']).size().reset_index(name='検収数')

# Extract only the day from the '検収日' column
kanban_grouped['検収日'] = pd.to_datetime(kanban_grouped['検収日']).dt.day

kanban_grouped.head()

,検収日,品番,検収数
0,1,019128GA010,19
1,1,01912ECB010,6
2,1,01912ECB040,18
3,1,1040043104R,1
4,1,1040052001Z,3


In [9]:
# Extract data for the selected product code
selected_kanban_product_data = kanban_grouped[kanban_grouped['品番'] == random_product_code]
selected_kanban_product_data

,検収日,品番,検収数
82,1,9030111A014,5
224,4,9030111A014,4
356,5,9030111A014,4
509,6,9030111A014,3
663,7,9030111A014,6
823,8,9030111A014,5
1000,11,9030111A014,5
1131,12,9030111A014,5
1254,13,9030111A014,4
1390,14,9030111A014,5


In [62]:
# Load the uploaded file with shift_jis encoding
arrangement_data = pd.read_csv('../data/202309_手配数_1Y_12次.csv', encoding='shift_jis')
arrangement_data.head()

,品番,加工図符号,設変符号,品名,ステータス,発注区分,整備室,整備室名,手配区分,工程内外製,...,21(木).1,22(金).1,23(土).1,24(日).1,25(月).1,26(火).1,27(水).1,28(木).1,29(金).1,30(土).1
0,35300-ECB010,NaN,NaN,"PUMP ASSY, OIL W/MOTOR",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,877,815,0,0,877,937,878,938,822,0
1,35580-6GA020,NaN,A,"ACTUATOR ASSY, SHIFT CONTROL",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,0,0,0,0,0,0,0,0,0,0
2,35580-ECB011,NaN,B,"ACTUATOR ASSY, SHIFT CONTROL",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,378,360,0,0,384,420,387,421,354,0
3,1040 052 001Z,NaN,D,"PLUG, W/HEAD STRAIGHT SCREW",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,499,455,0,0,493,517,491,517,468,0
4,1040 183 011P,NaN,C,"BOLT, FLANGE",使用中,かんばん,1Y,第１工場,手配,Ｐ,...,"1,134","1,080",0,0,"1,152","1,260","1,161","1,263","1,062",0


In [63]:
# Correctly format the date columns
date_columns_formatted = [f"{i}({day})" for i, day in zip(range(1, 31), "金土日月火水木" * 5)]
selected_columns_formatted = ['品番'] + date_columns_formatted
subset_data_formatted = arrangement_data[selected_columns_formatted]

# Melt the dataframe to long format
melted_data_formatted = pd.melt(subset_data_formatted, id_vars=['品番'], value_vars=date_columns_formatted, var_name='日付', value_name='日量数')

# Extract only the numeric part for '日付' column
melted_data_formatted['日付'] = melted_data_formatted['日付'].str.extract('(\d+)').astype(int)

# Extract the specified columns
additional_columns = ['当月必要数','当月稼働日数','仕入先名/工場名','箱種類', '収容数', '基準在庫日数', '基準在庫枚数', 'サイクル間隔', 'サイクル回数', 'サイクル情報']
additional_data = arrangement_data[['品番'] + additional_columns]

# Merge the additional data with the melted_data_formatted dataframe on '品番'
merged_data = pd.merge(melted_data_formatted, additional_data, on='品番', how='left')

# Rename the specified columns
merged_data.rename(columns={
    'サイクル間隔': 'A',
    'サイクル回数': 'B',
    'サイクル情報': 'C'
}, inplace=True)

# Convert column to numeric
merged_data['収容数'] = pd.to_numeric(merged_data['収容数'], errors='coerce')
merged_data['日量数'] = pd.to_numeric(merged_data['日量数'], errors='coerce')
merged_data['当月必要数'] = merged_data['当月必要数'].str.replace(",", "")
merged_data['当月必要数'] = pd.to_numeric(merged_data['当月必要数'], errors='coerce')
merged_data['当月稼働日数'] = pd.to_numeric(merged_data['当月稼働日数'], errors='coerce')

# Divide the '日量数' column by '収容数' and create a new column '日量数（箱）'
merged_data['日量数（箱）'] = merged_data['日量数'] / merged_data['収容数']

# Replace non-finite values with 0
merged_data['日量数（箱）'] = merged_data['日量数（箱）'].fillna(0)
# Round the '日量数（箱）' column and convert to integer
merged_data['日量数（箱）_切り上げ'] = np.ceil(merged_data['日量数（箱）']).astype(int)

merged_data['平均日量数（箱）'] = (merged_data['当月必要数'] /merged_data['当月稼働日数'])/merged_data['収容数']
# Replace non-finite values with 0
merged_data['平均日量数（箱）'] = merged_data['平均日量数（箱）'].fillna(0)

# Remove "-" and " " from the '品番' column in merged_data
merged_data['品番'] = merged_data['品番'].str.replace("-", "").str.replace(" ", "")

merged_data.head()

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）
0,35300ECB010,1,880.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12.0,0.91,3,1,4,8.04,73.333333,74,74.658730
1,355806GA020,1,0.0,0,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.70,3,1,4,8.04,0.000000,0,0.000000
2,35580ECB011,1,393.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.90,3,1,4,8.04,65.500000,66,65.357143
3,1040052001Z,1,487.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200.0,0.53,3,1,6,5.76,2.435000,3,2.518571
4,1040183011P,1,NaN,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.54,1,1,6,5.76,0.000000,0,2.352857


In [64]:
# Rename '検収日' column to '日付' in selected_kanban_product_data
kanban_grouped = kanban_grouped.rename(columns={'検収日': '日付'})

# Trim whitespace from the '品番' column in both dataframes
kanban_grouped['品番'] = kanban_grouped['品番'].str.strip()

# Merge merged_data and selected_kanban_product_data based on the specified conditions (品番 and 日付)
final_merged_data = pd.merge(merged_data, kanban_grouped, 
                                on=['品番', '日付'], how='inner')

final_merged_data.head()

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数
0,35300ECB010,1,880.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12.0,0.91,3,1,4,8.04,73.333333,74,74.658730,55
1,35580ECB011,1,393.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.90,3,1,4,8.04,65.500000,66,65.357143,49
2,1040052001Z,1,487.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200.0,0.53,3,1,6,5.76,2.435000,3,2.518571,3
3,1040183011P,1,NaN,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.54,1,1,6,5.76,0.000000,0,2.352857,2
4,3040052001B,1,NaN,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.52,3,1,6,5.76,0.000000,0,8.174667,6


In [65]:
# Rename '検収日' column to '日付' in selected_kanban_product_data
result = result.rename(columns={'計測日': '日付'})

# Trim whitespace from the '品番' column in both dataframes
result['品番'] = result['品番'].str.strip()

# Merge merged_data and selected_kanban_product_data based on the specified conditions (品番 and 日付)
final_merged_data = pd.merge(final_merged_data, result, 
                                on=['品番', '日付'], how='inner')
final_merged_data.head()

,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,基準在庫枚数,A,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）
0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12.0,0.91,3,1,4,8.04,78.833333,79,74.658730,50,32.0,41.0,127.0
1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.90,3,1,4,8.04,68.500000,69,65.357143,53,27.0,39.0,84.0
2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200.0,0.53,3,1,6,5.76,2.675000,3,2.518571,1,2.0,1.0,4.0
3,1040183011P,4,NaN,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.54,1,1,6,5.76,0.000000,0,2.352857,1,1.0,0.0,1.0
4,3040052001B,4,NaN,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.52,3,1,6,5.76,0.000000,0,8.174667,7,7.0,5.0,5.0


In [66]:
with open('..//data//順立装置の在庫の揺らぎ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    final_merged_data.to_csv(f)

In [67]:
# Try loading the uploaded file again using the Shift_JIS encoding
df1_new_attempt = pd.read_csv("../data/順立装置の在庫の揺らぎ.csv", encoding='shift_jis')
# Try loading the new uploaded file using the Shift_JIS encoding
df2_new_attempt = pd.read_csv("../data/不等ピッチ情報.csv", encoding='shift_jis')

In [68]:
# Merge df1_new_attempt with df2_new_attempt on the supplier name to add '不等ピッチ係数日' information without dropping columns
df1_updated = pd.merge(df1_new_attempt, df2_new_attempt[['仕入先名', '不等ピッチ係数日']], 
                       left_on='仕入先名/工場名', 
                       right_on='仕入先名', 
                       how='left')

# If '仕入先名' column exists in the merged dataframe, we drop it
if '仕入先名' in df1_updated.columns:
    df1_updated.drop(columns='仕入先名', inplace=True)

df1_updated.head()

,Unnamed: 0,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,...,B,C,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）,不等ピッチ係数日
0,0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12.0,0.91,...,4,8.04,78.833333,79,74.658730,50,32.0,41.0,127.0,0.14
1,1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.90,...,4,8.04,68.500000,69,65.357143,53,27.0,39.0,84.0,0.14
2,2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200.0,0.53,...,6,5.76,2.675000,3,2.518571,1,2.0,1.0,4.0,0.08
3,3,1040183011P,4,NaN,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.54,...,6,5.76,0.000000,0,2.352857,1,1.0,0.0,1.0,0.08
4,4,3040052001B,4,NaN,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.52,...,6,5.76,0.000000,0,8.174667,7,7.0,5.0,5.0,0.08


In [71]:
df1_updated['設計値MIN'] = 0.1 * (df1_updated['平均日量数（箱）'] * df1_updated['A'] * (1 +  df1_updated['C'])/ df1_updated['B'])
df1_updated['設計値MAX'] = df1_updated['設計値MIN'] + df1_updated['平均日量数（箱）']/df1_updated['B'] + df1_updated['平均日量数（箱）']*df1_updated['不等ピッチ係数日']
df1_updated.head()

,Unnamed: 0,品番,日付,日量数,当月必要数,当月稼働日数,仕入先名/工場名,箱種類,収容数,基準在庫日数,...,日量数（箱）,日量数（箱）_切り上げ,平均日量数（箱）,検収数,入庫数（箱）,出庫数（箱）,在庫数（箱）,不等ピッチ係数日,設計値MIN,設計値MAX
0,0,35300ECB010,4,946.0,18814,21.0,アイシン精機（株）,TP-341 ﾊﾝﾖｳ,12.0,0.91,...,78.833333,79,74.658730,50,32.0,41.0,127.0,0.14,16.872873,45.989778
1,1,35580ECB011,4,411.0,8235,21.0,アイシン精機（株）,TP-342 ｾﾝﾖｳ,6.0,0.90,...,68.500000,69,65.357143,53,27.0,39.0,84.0,0.14,14.770714,40.260000
2,2,1040052001Z,4,535.0,10578,21.0,（株）青山製作所,TP-131 ﾊﾝﾖｳ,200.0,0.53,...,2.675000,3,2.518571,1,2.0,1.0,4.0,0.08,0.283759,0.905007
3,3,1040183011P,4,NaN,24705,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.54,...,0.000000,0,2.352857,1,1.0,0.0,1.0,0.08,0.265089,0.845460
4,4,3040052001B,4,NaN,85834,21.0,（株）青山製作所,TP-331 ﾊﾝﾖｳ,500.0,0.52,...,0.000000,0,8.174667,7,7.0,5.0,5.0,0.08,0.921012,2.937430


In [70]:
with open('..//data//順立装置の在庫の揺らぎ.csv', mode='w',newline='', encoding='shift_jis',errors='ignore') as f:
    df1_updated.to_csv(f)